In [2]:
pip install Biopython

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
# import packages
import os
import regex as re
from Bio.Seq import MutableSeq
from Bio.Seq import reverse_complement
from Bio import SeqIO

In [4]:
def guide_locator(gRNA_len,target):
    #indexes starting pos of all PAM (NGG) sequences and then counts length of guide RNA upstream to locate starting pos of guides
    PAM_lo = [m.start(0) for m in re.finditer("[ATGC]GG",target, overlapped= True)]
    glo = [x - gRNA_len for x in PAM_lo] 
    PAM_lo
    glo

    # finds all possible guides
    poss_guides = []
    for i in range(len(PAM_lo)):
        if glo[i] >= 0:  #this is incase the PAM is too close to the beginning(guideRNA would be less than set length)
            poss_guides.append(target[glo[i]:PAM_lo[i]])
    return poss_guides


In [5]:
#RETURNS INPUT GUIDES ONLY - SHOULD GO INTO FINADALL
#GC Content monitor

def GC_monitor(guides,minGC,maxGC): 
    
    gc_filtered_dict ={}
    poor_gc ={}
    for x in guides:
        c = x.count("C")
        g = x.count("G")

        gc_total = g+c

        total=len(x)

        try:
            gc_content = gc_total/total
        except ZeroDivisionError:
            gc_content = 0

        if minGC < gc_content < maxGC:
            gc_filtered_dict[x]="{0:.2%}".format(gc_content)
        else:
            poor_gc[x]="{0:.2%}".format(gc_content)

    # input for aligner: #note: They are already gc filtered
    input_guides = list(gc_filtered_dict.keys())

    # count all guides rendered
    all_guides = (len(gc_filtered_dict) +  len(poor_gc)) 

    return input_guides

In [6]:
# START
# Load RepeatMasker annotations
with open("/home/milanamilan/design1_copy/sgRNA_design/repeatmasker.out", "r") as f:
    repeat_seq = f.readlines()

In [7]:
def fix_find(fasta_file,target_te,gRNA_len,minGC,maxGC):
    
    #maybe just add genome as input for this function and then parse w the parsing genome code after

    # Load and Parse human genome
    from Bio import SeqIO

    # Parse the FASTA file and store the records in a list
    records_dict = SeqIO.to_dict(SeqIO.parse(fasta_file, "fasta"))

    
    ##############

    #Find target sequences
    targets = []

    for line in repeat_seq[3:20]:
        fields = line.strip().split()

        for single_TE in target_te:

            if len(fields) > 13  and fields[9] == single_TE:
                start = int(fields[5])
                end = int(fields[6])
                strand = fields[8]

                for record_id, record in records_dict.items():
                    te_seq = (record.seq[start:end])
                    if strand == "-":
                        te_seq = te_seq.reverse_complement()
                if te_seq != '':
                    targets.append(str(te_seq)) if str(te_seq) not in targets else None #tabbing this line 2x makes the difference between 1 seq or 705

    #print (len(te_seq))
    #print (end-start)
    #print (te_seq)
    #print (len(targets))

    
    ##########


    for te in targets: 

        #Reverse-complement sequence
        rev_comp_seq = reverse_complement(te)
        (rev_comp_seq)

        #locate guides in both strands
        sense_guides= guide_locator(gRNA_len,target=te)
        anti_guides = guide_locator(gRNA_len,target=rev_comp_seq)

    #combine guides into one list
    unfiltered_guides = sense_guides + anti_guides
    
    #filters guides based on GC content
    x= GC_monitor(unfiltered_guides,minGC,maxGC)

    
    return targets

In [8]:
#this is the fix it code
fix_find("/home/milanamilan/design1_copy/sgRNA_design/hg38_chr19.fna",["LTR2B"],20,.4,.8)

: 

: 

In [ ]:
#result should be 6 different sequences